# Kili Tutorial: Importing assets

In this tutorial, we will walk through the process of using Kili to import assets. The goal of this tutorial is to illustrate some basic components and concepts of Kili in a simple way.

Additionally:

For an overview of Kili, visit https://kili-technology.com. You can also check out the Kili documentation https://cloud.kili-technology.com/docs.

The tutorial is divided into three parts:

1. Importing assets
2. Setting up a labeling priority
3. Setting up who can label the asset

This next cell connects the notebook to the Kili API. You need to update the credentials `api_key` before.

In [ ]:
import os
#!pip install kili
from kili.client import Kili

api_key = os.getenv('KILI_USER_API_KEY')
api_endpoint = os.getenv('KILI_API_ENDPOINT')
# If you use Kili SaaS, use the url 'https://cloud.kili-technology.com/api/label/v2/graphql'

kili = Kili(api_key=api_key, api_endpoint=api_endpoint)

In [ ]:
if 'recipes' in os.getcwd():
    os.chdir('..')

## Importing assets

An Asset is the smallest piece of information we want to label. It can be: a text, an image, a sound, etc...

To insert an asset in Kili, you need two essential pieces of information:

- The content of the asset (i.e. the text for a text document, or an url that points to the document, or the path to the image file on the hard drive).
- The name of the asset in Kili to be able to recognize it once inserted: `externalId`. **This name must be unique within the same project**.

It is also possible to associate **meta information** to certain assets that can be displayed in certain interfaces. 
The metadata takes the form of a dictionnary with three keys : `imageUrl`, `text` and `url`. You can see the metadata by clicking on a `!` on the labelling interface.
See below for examples and at the end of the recipe for a preview on Kili.

The cell below inserts three image files: two are in the form of image urls and one is in the form of a file on the hard drive. **Warning, before running this snippet, make sure you have cloned this repo, or have an image called man_on_a_bike.jpeg in a folder img**

In [ ]:
from tqdm import tqdm

CHUNK_SIZE = 100

def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]


assets = [
    {
        'externalId': 'example 1',
        'content': 'https://images.caradisiac.com/logos/3/8/6/7/253867/S0-tesla-enregistre-d-importantes-pertes-au-premier-trimestre-175948.jpg',
        'metadata': {"imageUrl": "https://i.pinimg.com/474x/f5/5e/95/f55e95f5677d2cb3bea922599667b6db--tesla-logo-car-logo-design.jpg", 
                     "text": "Tesla Company", 
                     "url": "https://www.tesla.com/fr_fr" },
        'toBeLabeledBy':['test@kili-technology.com']
    },
    {
        'externalId': 'example 2',
        'content': 'https://img.sportauto.fr/news/2018/11/28/1533574/1920%7C1280%7Cc096243e5460db3e5e70c773.jpg',
        'metadata': {"imageUrl": "https://upload.wikimedia.org/wikipedia/fr/thumb/e/e7/Logo_Porsche.svg/langfr-560px-Logo_Porsche.svg.png", 
                     "text": "Porsche Company", 
                     "url": "https://www.porsche.com/france/" },
        'priority': 1
    },
    {
        'externalId': 'example 3',
        'content': './recipes/img/man_on_a_bike.jpeg',
        'metadata': {},
    },
]
json_interface ={
    "jobs": {
        "CLASSIFICATION_JOB": {
            "mlTask": "CLASSIFICATION",
            "content": {
                "categories": {
                    "RED": {"name": "Red"},
                    "BLACK": {"name": "Black"},
                    "WHITE": {"name": "White"},
                    "GREY": {"name": "Grey"}},
                "input": "checkbox"
            },
            "required": 0,
            "isChild": False,
            "instruction": "Color"
        }
    }
}

project_id = kili.create_project(
        title='Project import assets',
        description='Import assets onto the app',
        input_type='IMAGE',
        json_interface=json_interface
)['id']

for asset_chunk in tqdm(list(chunks(assets, CHUNK_SIZE))):
    external_id_array = [a.get('externalId') for a in asset_chunk]
    content_array = [a.get('content') for a in asset_chunk]
    json_metadata_array = [a.get('metadata') for a in asset_chunk]
    kili.append_many_to_dataset(project_id=project_id, 
                                      content_array=content_array,
                                      external_id_array=external_id_array, 
                                      json_metadata_array=json_metadata_array)

The code provided also cuts into chunks the list of arrays to upload, as Kili only allows to upload 100 assets at a time onto the app. You can nevertheless such a loop structure to upload all your data.

## Setting up a labeling priority
In Kili, by default, the order in which the assets are labeled is the order in which they are inserted in Kili.

It is possible to change this order at any time, for example for active learning. To put an asset at the beginning of the stack, just change its `priority` property, which by default is 0, to a higher value. The higher the `priority` property, the higher the priority of the asset.

The cell below will put the asset `example 2` at the top of the stack.

In [ ]:
for asset in assets:
    if 'priority' not in asset:
        continue
    asset_id = kili.assets(external_id_contains=[asset.get('externalId')])[0]['id']
    kili.update_properties_in_assets(asset_ids=[asset_id], 
                                          priorities=[asset.get('priority')])

## Setting up who can label the asset
In some cases we also want to be able to choose who among the annotators will label the assets. To do this, simply pass to the asset the list of emails from the annotators in question with the `toBeLabeledBy` property.

The cell below will allow asset `example 1` to be only seen by test@kili-technology.com.

First we will need to add the user to our project.

In [ ]:
kili.append_to_roles(project_id=project_id, user_email='test@kili-technology.com', role='LABELER')

In [ ]:
for asset in assets:
    if 'toBeLabeledBy' not in asset:
        continue
    asset_id = kili.assets(project_id=project_id, external_id_contains=[asset.get('externalId')])[0]['id']
    kili.update_properties_in_assets(asset_ids=[asset_id], 
                                          to_be_labeled_by_array=[asset.get('toBeLabeledBy')])

![json metadata](./img/json_metadata.gif)

In this tutorial, we accomplished the following:

We introduced the concept of Kili asset content and asset external ID. We demonstrated how to append an asset to a project with the API.
We showed that an an asset can have a priority and a list a labelers.
If you enjoyed this tutorial, check out the other recipes for other tutorials that you may find interesting, including demonstrations of how to use Kili.

You can also visit the [Kili website](https://kili-technology.com) or [Kili documentation](https://cloud.kili-technology.com/docs) for more info!